In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train.head()

In [ ]:
trainimg = train.to_numpy()
trainimg.shape

In [ ]:
test_img = test.to_numpy().reshape(28000,28,28,1)
test_img.shape

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

train_x, train_y = train.to_numpy()[:,1:].reshape(42000,28,28,1), train.to_numpy()[:,0]

trainx, valx, trainy, valy = train_test_split(train_x, train_y, test_size=0.20)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
epoch = 10
model_his = model.fit(trainx, trainy, epochs=epoch)

In [ ]:
x = np.arange(1, epoch + 1)
train_acc = model_his.history['accuracy']
train_loss = model_his.history['loss']

In [ ]:
plt.plot(x, train_acc)
plt.show()
plt.plot(x, train_loss)
plt.show()

In [ ]:
# For predictions

prediction = model.predict(valx)
for i in range(len(prediction)):
    n = np.argmax(prediction[i,:])
    prediction[i,n] = 1
    prediction[i,:] = prediction[i,:].round()

In [ ]:
prediction.shape

In [ ]:
val_true = np.zeros((len(valy),10))

for i,j in enumerate(valy):
    val_true[i,j] = 1

In [ ]:
from sklearn.metrics import accuracy_score

my_accuracy = accuracy_score(val_true.astype('int32'), prediction.astype('int32'))
print('Accuracy:', my_accuracy)

In [ ]:
prediction_test = model.predict(test_img)
prediction_test.shape

In [ ]:
pred = np.zeros((28000,1))

for i in range(28000):
    pred[i] = np.argmax(prediction_test[i,:])

pred.shape

In [ ]:
ind = pd.DataFrame(np.arange(1,28001), columns=['ImageId'])

pred_sub = pd.DataFrame(pred.astype('int32'), columns=['Label'])
submission = [ind , pred_sub]
sub = pd.concat(submission, axis=1)

In [ ]:
compression_opts = dict(method='zip', archive_name='submission.csv')  

sub.to_csv('submission.zip', index=False, compression=compression_opts)

In [ ]:
pd.read_csv('submission.zip')